In [1]:
import pickle
import datetime
from dateutil.parser import parse

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np


import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#nltk.download()

In [2]:
# read data from pickle file
with open ('all_data_desk_nohole.pkl', 'rb') as fp:
    df = pickle.load(fp)
    
df.shape

(274606, 4)

In [ ]:
# there's some bad data in the snippets.  Drop the rows that have NaN to clean this up
df.dropna(inplace=True)

# take a subset of dataset to try things out
from sklearn.utils import shuffle
df2 = df.sample(n=20)
#df2 = shuffle(df2)

#df2 = df.copy()

# create an new dataframe that combines the headline and snippet (more words in document)

#df2['head_snip'] = df2['headline'].add(df2.snippet)
df2['head_snip'] = df['headline'] + ' '+ df['snippet']

dates = list(df2['date'])

df2.columns

In [ ]:
documents = list(df2['head_snip'])

In [ ]:
df2['head_snip'].to_csv('documents.csv')

## functions

In [ ]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print(f"Topic {topic_idx} (with {len(np.unique(W[:,topic_idx]))} articles)")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print('document with index',doc_index)
            print(documents[doc_index])
        print(' ')

In [ ]:
def plot_topic_trend_lda_old(H, W, documents, dates):
    plt.figure(figsize=(17,8))
    plot_data_all = []
    for topic_idx, topic in enumerate(H):
    #for topic_idx in range(0,1):
        count = 0
        print("Topic %d:" % (topic_idx))
        plot_data = {}
        foo = W[:,topic_idx]
        new_array = foo[np.where(foo != 0.)]
        print('documents in topic',len(np.unique(new_array)))
        
        for doc_index in np.argsort(new_array):
            #print(new_array[doc_index])
            if(new_array[doc_index] > 0.5):
                try:
                    #print(doc_index)
                    date_key = dates[int(doc_index)]
                    year, week, weekday = date_key.isocalendar()
                    if year == 2016:   # 2015 gets weeks 1-52, subsequent years have to add multiples of 52 to week number
                        week += 52
                    elif year == 2017:
                        week += 104
                    elif year == 2018:
                        week += 156

                    if week in plot_data:
                        plot_data[week] += 1
                    else:
                        plot_data[week] = 1
                except:
                    print(doc_index, 'in', topic_idx, 'had a problem')

        plt.plot(*zip(*sorted(plot_data.items())),label=f'Topic {topic_idx}')
        plot_data_all.append(plot_data)
    plt.legend(loc='upper left');
    return
#    return plot_data_all


In [ ]:
len(lda_W)

In [ ]:
min_date = min(dates)
max_date = max(dates)
min_year, min_week, weekday = min_date.isocalendar()
max_year, max_week, weekday = max_date.isocalendar()
max_year = 2015

plot_data_all = pd.DataFrame(columns=[['topic','date','num_docs']])
#plot_data_all['date'] = range(min_week,(max_year-min_year)*52+max_week+1)
w = 10
for topic in range(5):
    for week in range(10): 
        plot_data_all.at[w*topic+week,'topic'] = topic
        plot_data_all.at[w*topic+week,'date'] = week
        
plot_data_all['num_docs'] = 0
plot_data_all = plot_data_all.astype('int')
plot_data_all

In [ ]:
len(lda_W)

In [ ]:
plot_topic_trend_lda(lda_H,lda_W,documents,dates)

In [ ]:
import matplotlib.pyplot as plt

def plot_topic_trend_lda(H, W, documents, dates):
    plt.figure(figsize=(17,8))
    min_date = min(dates)
    max_date = max(dates)
    min_year, min_week, weekday = min_date.isocalendar()
    max_year, max_week, weekday = max_date.isocalendar()

    plot_data_all = pd.DataFrame(columns=[['topic','date','num_docs']])
    week_start = min_week
    week_end = (max_year-min_year)*52+max_week

    w = week_end - week_start
    for topic in range(len(H)):
        for week in range(week_start,week_end): 
            #print(w*topic+week, topic, week)
            plot_data_all.at[w*topic+week,'topic'] = topic
            plot_data_all.at[w*topic+week,'date'] = week
    plot_data_all['num_docs'] = 0
    plot_data_all = plot_data_all.astype('int')    
    
    #print(plot_data_all)

    for doc_idx in range(W.shape[0]):
        topic_idx = W[doc_idx].argmax()
        #print('doc_idx,topic_idx',doc_idx,topic_idx)
        #print('dates',dates[doc_idx])

        date_key = dates[doc_idx]
        #print('date key',date_key)
        year, week, weekday = date_key.isocalendar()
        #print('week',week)
        if year == 2016:   # 2015 gets weeks 1-52, subsequent years have to add multiples of 52 to week number
            week += 52
        elif year == 2017:
            week += 104
        elif year == 2018:
            week += 156
        #print('plot_data_all index',w*topic_idx+week, week)
        
        #bar = plot_data_all.at[w*topic_idx + week,'num_docs']
        #print('bar',bar)
        #plot_data_all.at[w*topic_idx + week,'num_docs'] = bar + 1
        bar = w*topic_idx + week
        plot_data_all.loc[bar,'num_docs'] +=  1
        
    plot_data_all.plot(x='date', y='num_docs', style='o')

  
    #print(plot_data_all)
#    plot_data = []
#    for index, row in plot_data_all.iterrows():
#        print([row['date'],row['num_docs']])
#        plot_data.append = [row['date'],row['num_docs']]
#    for topic in range(len(H)):
#        foo = plot_data_all[plot_data_all['topic'] == topic]
#        print(foo)
#        plt.plot(foo['date'],foo['num_docs'])
#    plt.legend(loc='upper left');
#    plt.plot(plot_data_all['date'],['num_docs'])
    return plot_data_all
#    return plot_data_all

def plot_topic_trend_nmf(H, W, documents, dates):
    plt.figure(figsize=(17,8))
    plot_data_all = []
    for topic_idx, topic in enumerate(H):
    #for topic_idx in range(0,1):
        count = 0
        print("Topic %d:" % (topic_idx))
        plot_data = {}
        foo = W[:,topic_idx]
        new_array = foo[np.where(foo != 0.)]
        print('documents in topic',len(np.unique(new_array)))

        for doc_index in np.argsort(new_array):
            try:
                #print(doc_index)
                date_key = dates[int(doc_index)]
                year, week, weekday = date_key.isocalendar()
                if year == 2016:
                    week += 52
                elif year == 2017:
                    week += 104
                elif year == 2018:
                    week += 156

                if week in plot_data:
                    plot_data[week] += 1
                else:
                    plot_data[week] = 1
            except:
                print(doc_index, 'in', topic_idx, 'had a problem')

        #plt.plot(*zip(*sorted(plot_data.items())),Ú)
        plot_data_all.append(plot_data)
    plt.legend(loc='upper left');
    #return
    return plot_data_all

## NMF

In [ ]:
no_features = 2000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, 
                                   min_df=2, 
                                   max_features=no_features, 
                                   stop_words='english')
#tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
print(len(tfidf_feature_names))

In [ ]:
tfidf_feature_names

In [ ]:
no_topics = 20

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

no_top_words = 10
no_top_documents = 2
display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)


In [ ]:
#plot_data_all = plot_topic_trend(nmf_H, nmf_W, tfidf_feature_names, documents, dates)
plot_topic_trend_nmf(nmf_H, nmf_W, documents, dates)

## LDA

In [ ]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
#tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
#tf = tf_vectorizer.fit_transform(documents)
tf_vectorizer = CountVectorizer(
            max_df = 0.95, # leave out words that occur in more than 95% of docs
            min_df = 2,    # leave out if occurs less than twice
            ngram_range=(1,3),
            stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
print(len(tf_feature_names))

In [ ]:
no_topics = 5

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.fit_transform(tf)
lda_H = lda_model.components_

In [ ]:
no_top_words = 10
no_top_documents = 5
display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)

In [ ]:
# write model to file (or read from file)

write = True # True if write to file
             # False if read from file
if write == True:
    # write data to pickle file
    with open('lda_model.pkl', 'wb') as fp:
        pickle.dump(lda_model, fp)
    with open('nmf_model.pkl', 'wb') as fp:
        pickle.dump(nmf_model, fp)

elif write == False:
    # read data from pickle file
    with open('lda_model.pkl', 'rb') as fp:
        lda_model = pickle.load(fp)
    with open('nmf_model_df95.pkl', 'rb') as fp:
        nmf_model = pickle.load(fp)


In [ ]:
'''
topic_idx = 4
foo = nmf_W[:,topic_idx]
new_array = foo[np.where(foo != 0.)]
for index in np.argsort(new_array):    
    print(index)
print(len(new_array))
'''

In [ ]:
for n in range(lda_W.shape[0]):
    topic_most_pr = lda_W[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))

In [ ]:
plot_topic_trend_lda(lda_H, lda_W, documents, dates)
#plot_data_all  = plot_topic_trend(lda_H, lda_W, documents, dates)

In [ ]:
# write data to csv file for reading into Tableau

outfile = open( 'plot_data_all_lda.csv', 'w' )
for i in range(len(plot_data_all)):
    for key, value in sorted( plot_data_all[:][i].items() ):
        outfile.write( str(i) + ',' + str(key) + ',' + str(value) + '\n' )
#        outfile.write( str(i) + '\t' + str(key) + '\t' + str(value) + '\n' )
#        print( str(i) + '\t' + str(key) + '\t' + str(value) + '\n' )
outfile.close()

In [ ]:
import pyLDAvis.gensim
import pyLDAvis.sklearn

import gensim
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer)

## using Word2Vect

In [ ]:
import os

# Setup nltk corpora path and Google Word2Vec location
google_vec_file = '/Users/dana/metis/GoogleNews-vectors-negative300.bin'

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [ ]:
headline = df2['headline'][0:10]
print(headline)

In [ ]:
print(stop_words)

In [ ]:
headline = df2['headline'][0:10]

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

for index, row in headline.iteritems():
    word_tokens = word_tokenize(row)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    stemmed_sentence = [ps.stem(w) for w in filtered_sentence]
    print(stemmed_sentence)#model.word_vec('word')

In [ ]:
for index, row in headline.iteritems():
    word_tokens = word_tokenize(row)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    #stemmed_sentence = [ps.stem(w) for w in filtered_sentence]
    stemmed_sentence = filtered_sentence
    print(stemmed_sentence)#model.word_vec('word')
    
    headline_vec = []

    for word in stemmed_sentence:
        try:
            vec = model.word_vec(word)
            headline_vec.append(vec)

        except:
            print('*not found*   ', word)

    print(len(headline_vec))
    print('')

In [ ]:
print(len(headline_vec))